In [7]:
import cv2
import numpy as np

In [8]:
cam = cv2.VideoCapture("resources/couple.mp4")

In [9]:
# Check if camera opened successfully
if (cam.isOpened()== False): 
  print("Error opening video stream or file")

# Displaying image frames per second
fps = cam.get(cv2.CAP_PROP_FPS)
print(f"The frame rate is: {fps}")

# define the number of frames per second
new_fps = 10

# determine fps scale factor
scale_fps = int(fps / new_fps)
print(f"new frame rate scale is: {scale_fps}")

The frame rate is: 30.0
new frame rate scale is: 3


In [10]:
# Create a window to hold the video frame
cv2.namedWindow("video", cv2.WINDOW_NORMAL)

# Create a list to hold the previously cropped images
prevKpPerReg = [[]] # This variable holds the previously found keypoints for the regions of interest
prevDesPerReg = [np.array([])] # this variable holds the previous found keypoint descriptions for the regions of interest


# The number of sample ROIs
sampleCount = 10

# tells us whether the first frame is the first
isFirstFrame = True


In [11]:
frameCount = 0
kp = [] #Holds the newly retrieved keypoints
des = [] # holds the newly retrieved keypoint descriptions
regionFrames = [] # Holds the various frames of the cropped regions

#looping through the video frames
while(cam.isOpened() == True):
    # get a video frame
    isRet, frame = cam.read()
   
    if(isRet == True & (frameCount%scale_fps == 0)):
        frameSectionToDisplay = np.array([])
        
        # Initiate ORB detector
        orb = cv2.ORB_create(1000)
        
        # initiate the brute force matcher
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # find the keypoints with ORB
        kp = orb.detect(frame,None)

        # compute the descriptors with ORB
        kp, des = orb.compute(frame, kp)
        
        
        if isFirstFrame == True:
            prevKpPerReg.clear() # Reset 
            prevDesPerReg.clear() # Reset
            
            # Extract the keypoints in the various regions
            for i in range(sampleCount):
                # Ending Y coordinate for the section
                maxY = int(frame.shape[0] / 2 + (i / sampleCount) * (frame.shape[0] / 2))
                # Starting Y coordinate for the section
                minY = int((i / sampleCount) * (frame.shape[0] / 2))
                _kps = []
                _des = np.array([])
                    
                for kp_item in  kp:
                    # find the image keypoints in this region
                    if kp_item.pt[0] < maxY and kp_item.pt[0] > minY:
                        _kps.append(kp_item)
                        np.append(_des, np.array([des[kp.index(kp_item)]]))
                
                prevKpPerReg.append(_kps)
                prevDesPerReg.append(_des)
            
            isFirstFrame = False
            continue
        
        else:
            # This variable stores the keypoints pertaining to each region
            currentKpPerReg = [[]] 
            # This variable stores the keypoint description for all keypoints in each region
            currentDesPerReg = [np.array([])]
            # Reset the region frames list
            regionFrames.clear()
            
            # Extracting key points for all regions
            for i in range(sampleCount):
                maxY = int(frame.shape[0] / 2 + (i / sampleCount) * (frame.shape[0] / 2))
                minY = int((i / sampleCount) * (frame.shape[0] / 2))
                regionFrames.append(frame[minY : maxY, : ])
                _kps = []
                _des = np.array([])
                    
                for kp_item in  kp:
                    # find the image keypoints in this region
                    if kp_item.pt[0] < maxY and kp_item.pt[0] > minY:
                        _kps.append(kp_item)
                        np.append(_des, np.array([des[kp.index(kp_item)]]))
                        
                currentKpPerReg.append(_kps)
                currentDesPerReg.append(_des)
            
            # find the region which change the most in the two frames
            minSimilarityScore = 1
            mostVaryingFrame = []
            
            for j in range(10):
                similarityScore = 1
                matches = bf.match(prevDesPerReg[j], currentDesPerReg[j])
                similar_regions = [i for i in matches if i.distance < 20]
                if len(matches) == 0 :
                    similarityScore = 0;
                else:
                    similarityScore = len(similar_regions) / len(matches)
                    
                if similarityScore < minSimilarityScore:
                    minSimilarityScore = similarityScore
                    mostVaryingFrame = regionFrames[j]
                    
            # Outdate the currentKpPerReg and currentDesPerReg Lists
            prevKpPerReg = currentKpPerReg
            prevDesPerReg = currentDesPerReg
            
                
            # newSampleFrame = frame[minY : maxY, : ]
            cv2.imshow(f"Cropped video", mostVaryingFrame)
            cv2.waitKey(5)


            # draw only keypoints location,not size and orientation
            frame = cv2.drawKeypoints(frame, kp, None, color=(0,255,0), flags=0)

            cv2.imshow("original video", frame)
        
    elif(isRet == False):
        break
    
    if (cv2.waitKey(20) & 0xFF == ord('q')):
        break;
    
# Increment the freme count
    frameCount += 1


In [12]:
# release resources
cam.release()
cv2.destroyAllWindows()